In [5]:
import pandas as pd
import pandas_gbq

from github import Github
from google.cloud import bigquery
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [6]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [7]:
year = 'year.2017'
query = """SELECT created_at, repo.name FROM githubarchive.{0} WHERE RAND() < 5/(SELECT COUNT(*) FROM `githubarchive.{0}`) ORDER BY created_at;""".format(year) 
project_id = 'cs515-term'

In [8]:
import pandas as pd
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)

repos = df['name']
dates = df['created_at']

Downloading: 100%|██████████| 10/10 [00:00<00:00, 57.30rows/s]


In [9]:
from github import Github, UnknownObjectException, GithubException
from tqdm import tqdm

with open('password', 'r') as file:
    password = file.read().replace('\n', '')

g = Github("siddhik97", password)
language_array = []

for repo in tqdm(repos):
    try:
        repo_details = g.get_repo(repo)
        if repo_details.language is not None:
            language_array.append(repo_details.language)
        else:
            language_array.append("NaN")
    except (UnknownObjectException, GithubException):
        language_array.append("NaN")
        pass

100%|██████████| 10/10 [00:01<00:00,  6.45it/s]
